In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict, namedtuple
from typing import Callable, Dict, List, Tuple
from trainer import Trainer, TrainerConfig
import jax
%matplotlib inline

In [4]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_from_single_image=True,
                           batch_size=8)
trainer_config = TrainerConfig(dataset_config=ds_config)

In [5]:
trainer = Trainer(trainer_config)
train_iter, val_iter = trainer.create_dataset()

In [6]:
next(train_iter)

Rays(origins=ShardedDeviceArray([[[ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ],
                     [ 0.8414927, -3.5839822,  2.186288 ]]],                   dtype=float32), directions=ShardedDeviceArray([[[ 0.08851627,  0.59432113,  0.7993322 ],
                     [ 0.5384322 ,  0.54934055,  0.6389854 ],
                     [-0.13287887,  0.80074024,  0.58407176],
                     [-0.51433915,  0.68965024,  0.5097269 ],
                     [-0.45183703,  0.6539291 ,  0.6067982 ],
                     [ 0.56656545,  0.45696044,  0.685696  ],
                     [ 0.1474538 ,  0.96288943,  0.22601214],
                     [ 0.16667055,

In [5]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in train_ds.take(200).as_numpy_iterator():
    origins = elements.origins
    directions = elements.directions
    pixels = elements.colors
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])

AttributeError: 'tuple' object has no attribute 'origins'

In [8]:
elements

ColoredRays(origins=array([[-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388]], dtype=float32), directions=array([[ 0.21775223,  0.36321905,  0.9058935 ],
       [ 0.48083252,  0.4632607 ,  0.74443144],
       [ 0.67416   ,  0.4551204 ,  0.5816901 ],
       [ 0.19054098,  0.37953192,  0.9053382 ],
       [ 0.5216486 ,  0.24016047,  0.8186518 ],
       [ 0.5905296 , -0.33727977,  0.7331424 ],
       [ 0.46349207,  0.44260776,  0.7676338 ],
       [ 0.3815262 , -0.24660194,  0.89084554]], dtype=float32), colors=array([[0.69411767, 0.6745098 , 0.69803923],
       [0.26666668, 0.30980393, 0.18431373],
       [0.30588236, 0.3137255 , 0.3019608 ],
       [0.654902  , 0.65882355, 0.6666667 ],
    

In [7]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [9]:
from read_write_model import read_model, qvec2rotmat

In [10]:
cameras_meta, images_meta, points = read_model("../dataset/pinecone/sparse/0/", ext=".bin")

In [15]:
cameras_meta[1].params

array([3.34624313e+03, 2.01600000e+03, 1.51200000e+03, 6.98396644e-03])

In [22]:
R = qvec2rotmat(images_meta[1].qvec)

In [24]:
from scipy.spatial.transform import Rotation as R

In [25]:
r = R.from_quat(images_meta[1].qvec)

In [33]:
r.as_matrix()

array([[ 0.49530899,  0.27762036, -0.82316216],
       [-0.22033811, -0.87642766, -0.42816547],
       [-0.84030954,  0.3934482 , -0.37293215]])

In [32]:
r.as_rotvec()

array([ 2.2577188 ,  0.04711942, -1.36834409])

In [21]:
images_meta[1]

Image(id=1, qvec=array([ 0.8283521 ,  0.01728801, -0.50204246,  0.24796631]), tvec=array([2.83913768, 0.04884715, 2.37824897]), camera_id=1, name='IMG_7238.JPG', xys=array([[1814.70959473,    5.92477465],
       [2477.25439453,    7.72096825],
       [ 710.68273926,   11.48809433],
       ...,
       [2755.18774414, 1459.08056641],
       [2755.18774414, 1459.08056641],
       [3617.06396484, 1057.89892578]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1]))